# GBN Evaluation
# Import packages and configurations

In [1]:
from core.model import GBNEncoder, LNClassifier, GBNDecoder
from core.sim_metric import EDSim, SDPSim, CosSim
import os
import torch

opt_encoder = {'data_dir': '../KnowledgeGraph_materials/data_kg/bootstrapnet_data/boot_pretrain_data_revised/',
       'output_model_file': './models/test', 
       'input_model_file': './models/210914_fineTuned_encoder', 
       'sim_metric': CosSim(), 'k_hop': 2, 
       'nl_weight': 0.01, 
       'local': False, 
       'n_layer': 3, 
       'dropout': 0.1, 
       'negative_slope': 0.2, 
       'bias': False, 
       'device': torch.device(type='cuda', index=0), 
       'cpu': False, 'seed': 1, 
       'n_epoch': 100, 
       'optimizer': 'adam', 
       'lr': 0.001, 
       'decay': 0.001, 
       'max_grad_norm': 1.0, 
       'feature_type': 'random', 
       'feature_dim': 50, 
       'edge_feature_dim': 5}


opt_decoder = {'dataset': '../KnowledgeGraph_materials/data_kg/bootstrapnet_data/boot_pretrain_data/gum_train/',
               'input_model_file': './models/210914_fineTuned_decoder',
               'output_model_file': '',
               'method': 'multi_view',
               'sim_metric': 'cos',
               'n_iter': 20,
               'min_match': 1,
               'n_expansion': 10,
               'k_hop': 2,
               'un_weight': 0.01,
               'local': False,
               'mean_updated': False,
               'n_layer': 3,
               'dropout': 0.1,
               'negative_slope': 0.2,
               'bias': False,
               'device': torch.device(type='cuda', index=0),
               'cpu': False,
               'seed': 1,
               'init_encoder_epoch': 200,
               'init_decoder_epoch': 200,
               'encoder_epoch': 50,
               'decoder_epoch': 50,
               'optimizer': 'adam',
               'lr': 0.001,
               'decay': 0.001,
               'max_grad_norm': 1.0,
               'feature_type': 'glove',
               'feature_dim': 50,
               'edge_feature_dim': 5}


# Load encoder & decoder

In [3]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html

encoder = GBNEncoder(opt_encoder)

encoder.load_state_dict(torch.load(opt_encoder['input_model_file'] +'.pth'))

decoder = GBNDecoder(opt_decoder, opt_decoder['sim_metric'])

decoder.load_state_dict(torch.load(opt_decoder['input_model_file'] +'.pth'))

# print(encoder.eval())
# print(decoder.eval())

<All keys matched successfully>

# Predict links by encoder 